In [13]:
from pathlib import Path
import os
import numpy as np
import torch
import time
import matplotlib.pyplot as plt

import myextensions
from utils import visualize_cam, Normalize
from gradcam import GradCAM, GradCAMpp

In [14]:
torch.manual_seed(123)
np.random.seed(123)

model = myextensions.get_vgg()
model.eval(), model.cuda()

cam_dict = dict()

vgg_model_dict = dict(type='vgg', arch=model, layer_name='features_29', input_size=(224, 224))

gradcam = GradCAMpp(vgg_model_dict, True)

import warnings
warnings.filterwarnings("ignore", category=FutureWarning, module="torch.nn.modules.module")

saliency_map size : torch.Size([14, 14])


In [15]:
def save_3_heatmaps(dATASET_PATH, filename, heatmap, input_image):
    filepath_pos = os.path.join(dATASET_PATH, "pos", filename)
    filepath_neg = os.path.join(dATASET_PATH, "neg", filename)
    filepath_all = os.path.join(dATASET_PATH, "all", filename)
    filepath_blended = os.path.join(dATASET_PATH, "blended", filename)

    vmin, vmax = -1, 1
    plt.ioff()

    fig, plt_axis = plt.subplots(figsize=(7, 7))
    plt_axis.xaxis.set_ticks_position("none")
    plt_axis.yaxis.set_ticks_position("none")
    plt_axis.set_yticklabels([])
    plt_axis.set_xticklabels([])
    plt_axis.grid(visible=False)
    _ = plt_axis.imshow(heatmap, cmap='bwr', alpha=0.7, vmin=vmin, vmax=vmax)

    fig.savefig(filepath_all, bbox_inches='tight', pad_inches=0)

    _ = plt_axis.imshow(np.mean(input_image, axis=2), cmap="gray")
    _ = plt_axis.imshow(heatmap, cmap='bwr', alpha=0.7, vmin=vmin, vmax=vmax)

    fig.savefig(filepath_blended, bbox_inches='tight', pad_inches=0)

    plt.close()
    

def print_time(savepath,timings, filenum):
    timings_np = np.array(timings)
    with open(f'{savepath}/time_statistics{filenum}.txt', "w") as f:
        f.write("=== Attribution Time Stats ===\n")
        f.write(f"Total images:      {len(timings)}\n")
        f.write(f"Average time:      {timings_np.mean():.4f} s\n")
        f.write(f"Standard deviation:{timings_np.std():.4f} s\n")
        f.write(f"Minimum time:      {timings_np.min():.4f} s\n")
        f.write(f"Maximum time:      {timings_np.max():.4f} s\n")

In [16]:
timings = []

MODES = \
[
    "adversial-patch/success", "adversial-patch/fail",
    "feature-adversaries/success", "feature-adversaries/fail",
    # "fgsm/success", "fgsm/fail",
    # "no-attack", 
    # "pgd/success", "pgd/fail",
    # "shadow-attack-nontargeted/success", "shadow-attack-nontargeted/fail",
    # "square-attack-l2/success", "square-attack-l2/fail",
    # "square-attack-linf/success", "square-attack-linf/fail",
]




for mode in MODES:
    dATASET_PATH = f"../../inputs/{mode}"
    sAVE_DIR_PATH = f"/home/cat/uni/bakis/outputs/gradcam++/{mode}"

    Path(f"{sAVE_DIR_PATH}/pos").mkdir(parents=True, exist_ok=True)
    Path(f"{sAVE_DIR_PATH}/neg").mkdir(parents=True, exist_ok=True)
    Path(f"{sAVE_DIR_PATH}/all").mkdir(parents=True, exist_ok=True)
    Path(f"{sAVE_DIR_PATH}/blended").mkdir(parents=True, exist_ok=True)

    sorted_filenames = os.listdir(dATASET_PATH)
    sorted_filenames.sort()

    for filename in sorted_filenames:
        full_path = os.path.join(dATASET_PATH, filename)

        input_image, input_image_np = myextensions.get_image(full_path, myextensions.PREPROCESS_ATTACK)

        pred_label_idx, predicted_label, prediction_score = myextensions.get_prediction(model, input_image, topk = 5)

        input_image.requires_grad_()

        start_time = time.perf_counter()
        
        mask, _ = gradcam(input_image)

        elapsed = time.perf_counter() - start_time
        
        timings.append(elapsed)

        try:
            save_3_heatmaps(
                sAVE_DIR_PATH,
                filename, 
                mask.squeeze().cpu().numpy(),
                np.transpose(input_image_np.squeeze(), (1, 2, 0)))
        except Exception:
            print(f"Exception with: {full_path}")
            pass

    if (timings != []):
        print_time(sAVE_DIR_PATH ,timings, 0)
        timings = []